# ENTREGABLE 1 - EXTRAER DATOS DE API DE YOUTUBE E INSERTAR LA DATA EN UNA TABLA DE AWS REDSHIFT.

El código siguiente obtiene los 10 videos más populares (con más vistas) de YouTube publicados después de la fecha actual. Utiliza la API de YouTube para realizar una búsqueda de videos, ordenándolos por la cantidad de vistas (ordenados en orden descendente). Luego, obtiene los detalles de esos videos y crea un DataFrame con la información relevante, incluida la cantidad de vistas, y lo imprime.

Si deseara obtener los 10 videos con la mayor cantidad de vistas en un día específico, puedes modificar la variable current_date en el código para que coincida con la fecha que deseas. Por ejemplo:

#Cambiar esta línea según la fecha que quiera:

current_date = '2023-08-23T00:00:00Z'

In [168]:
# Guardo mi contraseña "api_key" de la API de Youtuber en un archivo .txt por cuestiones de seguridad y que no aparezca visible mi contraseña en el código:
with open("C:/Users/cnieto1/Desktop/Curso Data Engineering - Coderhouse/Clases/Entregable 1/api_key_youtube.txt",'r') as f:
    pwd= f.read()

In [169]:
import pandas as pd
from googleapiclient.discovery import build
import datetime

# Definir tu clave de API de YouTube
API_KEY = pwd

# Crear una instancia del servicio de la API de YouTube
youtube = build('youtube', 'v3', developerKey=API_KEY)

# Función para obtener el nombre de la categoría a partir del ID
def get_category_name(youtube, category_id):
    categories_response = youtube.videoCategories().list(
        part='snippet',
        id=category_id
    ).execute()
    if 'items' in categories_response:
        return categories_response['items'][0]['snippet']['title']
    else:
        return 'Desconocida'

# Función para convertir la duración en formato "PT11M13S" a segundos
def convert_duration_to_seconds(duration):
    parts = duration[2:].split('T')[-1].split('H')
    hours = int(parts[0]) if len(parts) > 1 else 0
    minutes_parts = parts[-1].split('M')
    minutes = int(minutes_parts[0]) if len(minutes_parts) > 1 else 0
    seconds_parts = minutes_parts[-1].split('S')
    seconds = int(seconds_parts[0]) if len(seconds_parts) > 1 else 0
    total_seconds = hours * 3600 + minutes * 60 + seconds
    return total_seconds

# Obtener la fecha actual en el formato requerido por la API de YouTube
current_date = datetime.datetime.now().strftime('%Y-%m-%dT00:00:00Z')

# Realizar la búsqueda de videos ordenados por vistas y limitar a 10 resultados
search_response = youtube.search().list(
    part='id',
    maxResults=10,
    order='viewCount',
    type='video'
).execute()

# Extraer los IDs de los videos obtenidos en la búsqueda
video_ids = [item['id']['videoId'] for item in search_response['items']]

# Obtener detalles de los videos
videos_response = youtube.videos().list(
    part='snippet,statistics,contentDetails',
    id=','.join(video_ids)
).execute()

# Crear una lista de diccionarios con la información de los videos
video_data = []
for video in videos_response['items']:
    video_id = video['id']
    video_info = {
        "ID_del_Video": video_id,
        "Título": video['snippet']['title'],
        "Descripción": video['snippet']['description'],
        "Canal_Propietario": video['snippet']['channelTitle'],
        "Fecha_de_Publicación": video['snippet']['publishedAt'],
        "Categoría_ID": video['snippet']['categoryId'],
        "Categoría": get_category_name(youtube, video['snippet']['categoryId']),
        "Duración_segundos": convert_duration_to_seconds(video['contentDetails']['duration']),
        "URL_del_Video": f"https://www.youtube.com/watch?v={video_id}",      
        "Vistas": video['statistics']['viewCount'],
        "Likes": video['statistics'].get('likeCount', 0),
        "Dislikes": video['statistics'].get('dislikeCount', 0),
        "Favorite_Count": video['statistics'].get('favoriteCount', 0),
        "Comment_Count": video['statistics'].get('commentCount', 0),
        "Insert_Date": current_date
    }
    video_data.append(video_info)

# Crear un DataFrame a partir de la lista de diccionarios
df = pd.DataFrame(video_data)

# Mostrar el DataFrame
print(df)

  ID_del_Video                                             Título  \
0  0aZ7lPQ5EXs  El Gallo y la Pata - Canciones de la Granja de...   
1  eNLjdPI9zdE   La Vaca Lola - Canciones de La Granja de Zenón 2   
2  ebVVuJN1WFM                   Bartolito - La Granja de Zenón 3   
3  bX3S-_jUauc  Paulo Londra ft Lenny Tavarez - Nena Maldicion...   
4  aSjflT_J0Xo         Paulo Londra - Adan y Eva (Official Video)   
5  4nKcnfw9ggc                   Percherón - La Granja de Zenón 3   
6  NPpELzyP4rw            Paulo Londra - Tal Vez (Official Video)   
7  KATq-Ws3xtM  Patitos Cua Cua Cua - Canciones y clásicos inf...   
8  q8dilxHvbiM  Soy una Serpiente - Las Canciones del Zoo 4 | ...   
9  nBANlLaki_Y  Cuidado con la Bomba Chita - Las Canciones del...   

                                         Descripción   Canal_Propietario  \
0  🎁 En estas Navidades, encuentra los productos ...   El Reino Infantil   
1  🎁 En estas Navidades, encuentra los productos ...   El Reino Infantil   
2  🎁 En esta

In [170]:
df.head()

,ID_del_Video,Título,Descripción,Canal_Propietario,Fecha_de_Publicación,Categoría_ID,Categoría,Duración_segundos,URL_del_Video,Vistas,Likes,Dislikes,Favorite_Count,Comment_Count,Insert_Date
0,0aZ7lPQ5EXs,El Gallo y la Pata - Canciones de la Granja de...,"🎁 En estas Navidades, encuentra los productos ...",El Reino Infantil,2014-03-15T14:30:00Z,10,Music,144,https://www.youtube.com/watch?v=0aZ7lPQ5EXs,1912305769,4555633,0,0,0,2023-08-23T00:00:00Z
1,eNLjdPI9zdE,La Vaca Lola - Canciones de La Granja de Zenón 2,"🎁 En estas Navidades, encuentra los productos ...",El Reino Infantil,2014-04-12T14:30:01Z,10,Music,144,https://www.youtube.com/watch?v=eNLjdPI9zdE,1820654493,5103162,0,0,0,2023-08-23T00:00:00Z
2,ebVVuJN1WFM,Bartolito - La Granja de Zenón 3,"🎁 En estas Navidades, encuentra los productos ...",La Granja de Zenón,2015-11-14T13:00:01Z,10,Music,148,https://www.youtube.com/watch?v=ebVVuJN1WFM,1267195524,2876258,0,0,0,2023-08-23T00:00:00Z
3,bX3S-_jUauc,Paulo Londra ft Lenny Tavarez - Nena Maldicion...,"Paulo Londra ""Nena Maldicion""\n\n(Apple Music)...",Paulo Londra,2018-01-29T22:00:05Z,10,Music,232,https://www.youtube.com/watch?v=bX3S-_jUauc,1238108268,6590520,0,0,250794,2023-08-23T00:00:00Z
4,aSjflT_J0Xo,Paulo Londra - Adan y Eva (Official Video),"Paulo Londra ""Adan y Eva""\n\n(Apple) ▶ https:...",Paulo Londra,2018-11-05T18:00:04Z,10,Music,261,https://www.youtube.com/watch?v=aSjflT_J0Xo,1220381369,7282776,0,0,279526,2023-08-23T00:00:00Z


In [171]:
df

,ID_del_Video,Título,Descripción,Canal_Propietario,Fecha_de_Publicación,Categoría_ID,Categoría,Duración_segundos,URL_del_Video,Vistas,Likes,Dislikes,Favorite_Count,Comment_Count,Insert_Date
0,0aZ7lPQ5EXs,El Gallo y la Pata - Canciones de la Granja de...,"🎁 En estas Navidades, encuentra los productos ...",El Reino Infantil,2014-03-15T14:30:00Z,10,Music,144,https://www.youtube.com/watch?v=0aZ7lPQ5EXs,1912305769,4555633,0,0,0,2023-08-23T00:00:00Z
1,eNLjdPI9zdE,La Vaca Lola - Canciones de La Granja de Zenón 2,"🎁 En estas Navidades, encuentra los productos ...",El Reino Infantil,2014-04-12T14:30:01Z,10,Music,144,https://www.youtube.com/watch?v=eNLjdPI9zdE,1820654493,5103162,0,0,0,2023-08-23T00:00:00Z
2,ebVVuJN1WFM,Bartolito - La Granja de Zenón 3,"🎁 En estas Navidades, encuentra los productos ...",La Granja de Zenón,2015-11-14T13:00:01Z,10,Music,148,https://www.youtube.com/watch?v=ebVVuJN1WFM,1267195524,2876258,0,0,0,2023-08-23T00:00:00Z
3,bX3S-_jUauc,Paulo Londra ft Lenny Tavarez - Nena Maldicion...,"Paulo Londra ""Nena Maldicion""\n\n(Apple Music)...",Paulo Londra,2018-01-29T22:00:05Z,10,Music,232,https://www.youtube.com/watch?v=bX3S-_jUauc,1238108268,6590520,0,0,250794,2023-08-23T00:00:00Z
4,aSjflT_J0Xo,Paulo Londra - Adan y Eva (Official Video),"Paulo Londra ""Adan y Eva""\n\n(Apple) ▶ https:...",Paulo Londra,2018-11-05T18:00:04Z,10,Music,261,https://www.youtube.com/watch?v=aSjflT_J0Xo,1220381369,7282776,0,0,279526,2023-08-23T00:00:00Z
5,4nKcnfw9ggc,Percherón - La Granja de Zenón 3,Ya puedes usar los filtros de La Vaca Lola en ...,El Reino Infantil,2015-11-21T12:00:01Z,10,Music,132,https://www.youtube.com/watch?v=4nKcnfw9ggc,1111412960,2600919,0,0,0,2023-08-23T00:00:00Z
6,NPpELzyP4rw,Paulo Londra - Tal Vez (Official Video),"Paulo Londra ""Tal Vez""\n(Apple) ▶ https://appl...",Paulo Londra,2019-04-03T22:00:02Z,10,Music,272,https://www.youtube.com/watch?v=NPpELzyP4rw,1078655843,6962901,0,0,212333,2023-08-23T00:00:00Z
7,KATq-Ws3xtM,Patitos Cua Cua Cua - Canciones y clásicos inf...,🎉¡Bienvenido a El Reino Infantil en Español! A...,El Reino Infantil,2014-12-06T12:30:03Z,10,Music,117,https://www.youtube.com/watch?v=KATq-Ws3xtM,1023668570,1643181,0,0,0,2023-08-23T00:00:00Z
8,q8dilxHvbiM,Soy una Serpiente - Las Canciones del Zoo 4 | ...,"🎁 En estas Navidades, encuentra los productos ...",El Reino Infantil,2018-03-03T11:59:53Z,10,Music,136,https://www.youtube.com/watch?v=q8dilxHvbiM,882845620,1973413,0,0,0,2023-08-23T00:00:00Z
9,nBANlLaki_Y,Cuidado con la Bomba Chita - Las Canciones del...,Suscríbete ► http://bit.ly/SuscribeteElReinoIn...,El Reino Infantil,2013-11-30T14:30:01Z,10,Music,147,https://www.youtube.com/watch?v=nBANlLaki_Y,837219844,1148657,0,0,0,2023-08-23T00:00:00Z


In [172]:
#Obtenemos un Dataframe de 10 registros y 15 columnas:
df.shape

(10, 15)

In [173]:
#Hago las siguientes transformaciones a las columnas del Dataframe df:

# Recortar la columna "Descripción" y "Título" a 301 caracteres:
df['Descripción'] = df['Descripción'].str[:301]
df['Título'] = df['Título'].str[:301]

import pandas as pd
from datetime import datetime

# Convertir la columna "Fecha de Publicación" y "Insert Date" a objetos datetime:
df['Fecha_de_Publicación'] = pd.to_datetime(df['Fecha_de_Publicación'])
df['Insert_Date'] = pd.to_datetime(df['Insert_Date'])

# Formatear la columna "Fecha de Publicación" y "Insert Date" en el formato deseado:
df['Fecha_de_Publicación'] = df['Fecha_de_Publicación'].dt.strftime('%Y-%m-%d')
df['Insert_Date'] = df['Insert_Date'].dt.strftime('%Y-%m-%d')

df.head()

,ID_del_Video,Título,Descripción,Canal_Propietario,Fecha_de_Publicación,Categoría_ID,Categoría,Duración_segundos,URL_del_Video,Vistas,Likes,Dislikes,Favorite_Count,Comment_Count,Insert_Date
0,0aZ7lPQ5EXs,El Gallo y la Pata - Canciones de la Granja de...,"🎁 En estas Navidades, encuentra los productos ...",El Reino Infantil,2014-03-15,10,Music,144,https://www.youtube.com/watch?v=0aZ7lPQ5EXs,1912305769,4555633,0,0,0,2023-08-23
1,eNLjdPI9zdE,La Vaca Lola - Canciones de La Granja de Zenón 2,"🎁 En estas Navidades, encuentra los productos ...",El Reino Infantil,2014-04-12,10,Music,144,https://www.youtube.com/watch?v=eNLjdPI9zdE,1820654493,5103162,0,0,0,2023-08-23
2,ebVVuJN1WFM,Bartolito - La Granja de Zenón 3,"🎁 En estas Navidades, encuentra los productos ...",La Granja de Zenón,2015-11-14,10,Music,148,https://www.youtube.com/watch?v=ebVVuJN1WFM,1267195524,2876258,0,0,0,2023-08-23
3,bX3S-_jUauc,Paulo Londra ft Lenny Tavarez - Nena Maldicion...,"Paulo Londra ""Nena Maldicion""\n\n(Apple Music)...",Paulo Londra,2018-01-29,10,Music,232,https://www.youtube.com/watch?v=bX3S-_jUauc,1238108268,6590520,0,0,250794,2023-08-23
4,aSjflT_J0Xo,Paulo Londra - Adan y Eva (Official Video),"Paulo Londra ""Adan y Eva""\n\n(Apple) ▶ https:...",Paulo Londra,2018-11-05,10,Music,261,https://www.youtube.com/watch?v=aSjflT_J0Xo,1220381369,7282776,0,0,279526,2023-08-23


In [174]:
# Guardo mi contraseña "pwd_redshift" de mi cuenta de Redshift en un archivo .txt por cuestiones de seguridad y que no aparezca visible mi contraseña en el código:
with open("C:/Users/cnieto1/Desktop/Curso Data Engineering - Coderhouse/Clases/Entregable 1/redshift_password.txt",'r') as f:
    pwd_redshift= f.read()

In [175]:
# Creando la conexión a Redshift:
import psycopg2
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="christian_r_coderhouse"

try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password= pwd_redshift,
        port='5439'
    )
    print("Conectado a Redshift con éxito!")
    
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


In [176]:
# Código para hacer Drop Table de la tabla "canciones". USARLO SOLO en caso de que la tabla ya exista y tenga que hacerle modificaciones:
# RECORDAR: Antes de correr este código, correr primero el código anterior (Creando la conexión a Redshift):

# Crear un cursor:
#cur = conn.cursor()

# Ejecutar la sentencia DROP TABLE:
#cur.execute("DROP TABLE IF EXISTS videos")

# Hacer commit para aplicar los cambios:
#conn.commit()

In [177]:
#Crear la tabla si no existe:
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS videos
        (
        Id_del_Video VARCHAR(50) primary key
        ,Título VARCHAR(350)
        ,Descripción VARCHAR(350)   
        ,Canal_Propietario VARCHAR(255)
        ,Fecha_de_Publicación date
        ,Categoría_ID VARCHAR(50)
        ,Categoría VARCHAR(100)
        ,Duración_segundos INTEGER
        ,URL_del_Video NVARCHAR(500)
        ,Vistas INTEGER
        ,Likes INTEGER
        ,Dislikes INTEGER
        ,Favorite_Count INTEGER
        ,Comment_Count INTEGER
        ,Insert_Date date
      
        )
    """)
    conn.commit()

In [178]:
# Comento este paso, para que cada vez que corra el script de la API, que me vaya insertando los registros de cada día y se vayan acumulando:
#Vaciar la tabla para evitar duplicados o inconsistencias:
#with conn.cursor() as cur:
#  cur.execute("Truncate table canciones")
#  count = cur.rowcount
# count


In [179]:
#consultando la tabla canciones:
cur = conn.cursor()
cur.execute("SELECT * FROM videos")
results = cur.fetchall()
#results

In [180]:
#Insertando los datos en Redsfhift:
from psycopg2.extras import execute_values
with conn.cursor() as cur:
    execute_values(
        cur,
        '''
        INSERT INTO videos (ID_del_Video, Título, Descripción, Canal_Propietario, Fecha_de_Publicación, Categoría_ID, Categoría, Duración_segundos, URL_del_Video, Vistas, Likes, Dislikes, Favorite_Count, Comment_Count, Insert_Date)
        VALUES %s
        ''',
        [tuple(row) for row in df.values],
        page_size=len(df)
    )
    conn.commit()

In [181]:
# Veo cómo quedó la tabla en Redshift luego de hacer los Insert:
#consultando la tabla
cur = conn.cursor()
cur.execute("SELECT * FROM videos")
results = cur.fetchall()

In [182]:
# Veo cómo quedó la tabla "canciones" en Redshift. Convierto "results" al DataFrame "df_redshift":
column_names=['Video_ID', 'Title', 'Description', 'Channel', 'Publication_Date', 'Category_ID', 'Category','Duration_seconds','Video_URL', 'Views_Count','Likes', 'Dislikes','Favorite_Count','Comment_Count','Insert_date']
df_redshift= pd.DataFrame(results, columns=column_names)
df_redshift.head()

,Video_ID,Title,Description,Channel,Publication_Date,Category_ID,Category,Duration_seconds,Video_URL,Views_Count,Likes,Dislikes,Favorite_Count,Comment_Count,Insert_date
0,0aZ7lPQ5EXs,El Gallo y la Pata - Canciones de la Granja de...,"🎁 En estas Navidades, encuentra los productos ...",El Reino Infantil,2014-03-15,10,Music,144,https://www.youtube.com/watch?v=0aZ7lPQ5EXs,1912305769,4555629,0,0,0,2023-08-23
1,eNLjdPI9zdE,La Vaca Lola - Canciones de La Granja de Zenón 2,"🎁 En estas Navidades, encuentra los productos ...",El Reino Infantil,2014-04-12,10,Music,144,https://www.youtube.com/watch?v=eNLjdPI9zdE,1820654493,5103154,0,0,0,2023-08-23
2,ebVVuJN1WFM,Bartolito - La Granja de Zenón 3,"🎁 En estas Navidades, encuentra los productos ...",La Granja de Zenón,2015-11-14,10,Music,148,https://www.youtube.com/watch?v=ebVVuJN1WFM,1267194388,2876256,0,0,0,2023-08-23
3,bX3S-_jUauc,Paulo Londra ft Lenny Tavarez - Nena Maldicion...,"Paulo Londra ""Nena Maldicion""\n\n(Apple Music)...",Paulo Londra,2018-01-29,10,Music,232,https://www.youtube.com/watch?v=bX3S-_jUauc,1238108268,6590520,0,0,250794,2023-08-23
4,aSjflT_J0Xo,Paulo Londra - Adan y Eva (Official Video),"Paulo Londra ""Adan y Eva""\n\n(Apple) ▶ https:...",Paulo Londra,2018-11-05,10,Music,261,https://www.youtube.com/watch?v=aSjflT_J0Xo,1220380842,7282774,0,0,279526,2023-08-23


In [183]:
# Cierro tanto el cursor como la conexión a la base de datos:
cur.close()
conn.close()